## Common Patterns in DP
### Iteration in the recurence relation
* in Min Cost Climbing Stairs problem (Leetcode 746), we are allowed to climb one or two stairs each time, what if we can climb up to k steps?
  + instead of dp(i) = min(dp(i-1) + cost\[i-1\], dp(i-2) + cost\[i-2\]), we have
  + dp(i) = min(dp(j) + cost\[j\]) for all (i-k) <= j< i
  + this is a common pattern in DP problems
* implementation
  + usually add a for-loop to iterate through a dynamic number of options and choose the best one

### Leetcode 1335 Minimum Difficulty of a Job Schedule
* overview
  + you are given a list of jobDifficulty, each element corresponds to a difficulty value of a job. 
  + you are given a integer d, which defines in how many days, you should schedule all the jobs in jobDifficulty list
  + for each day you schedule jobs, the job difficulty of that day is the max of the jobs in that day
  + find the sechdule corresponding to the mninimum of the total difficulties of all the days and return the total
* implementation of top down
  + This is a DP problem, because the choice of each day affects the later day's schedule and the final resutls
    + two state variables, one is to define the starting index of the current day, the second is how many days are remaining
    + if the remaining day is zero, returns -1
    + if the remaining day is one, returns the max of the remaining job difficulties
    + for each function schedule(index, remain_day), it checks all the possibility of taking one to n-remain_day+1 jobs where n is the total number of jobs, translating to the index is current index to n-remaining_day (remember, index is 0 based). For example, if the current index is 2, and we have 3 remaining days, with totally 5 tasks, then the last index we can get to is 2, so `for i in range(index, n-remaining_day +1)` here index = 2, n=5, remaining_day = 3, so we have the for loop execute once to just pick up one task, since the two remaining tasks (indexed at 3 and 4) will need to be assigned to the other 2 reamining days
  + To avoid the TLE, set the maxsize of lru_cache to be None, so that the cache size will increase indefinitely with the entries
  + Therefore, it is a correct implementation, but due to the recursive stack, the runtime performance is not good enough
* implementation of bottom up 
  + Same thing to top down to build the max_arr that contains the max element starting from the corresponding index to the end of the array
  + we initialize a 2D DP array with n rows and d+1 columns. Each element has the default value of -1, which is the return value if schedule is not possible. 
    + assigning default values to -1 is not necessary, since the no-answer cases have been excluded by the first statement 
  + Each row refers to a starting index of the jobs, and each column refer to the day from which to schedule the job. For example, dp(0)(3) means to schedule jobs starting from the first job (index = 0) from day 3. 
    + note that we don't use day 0, we use day columns from day 1 to day == d, which is the last day. This is just for convenience to organize code and count days
    + The final results will be dp(0)(1) meaning the minimum total job difficulty to schedule starting from the first job and the first day
  + in DP, for each day's schedule, the starting job index is restricted to be in the range of d-1 to n-1. This is because for a specific day, we have to make sure there are jobs assigned to all the days before it, at least one job for each of the previous days. Therefore, for day 2, the earlies starting index will be 1, since job 0 has to be assigned to day 1
  + we first implemen the base case of the last column (column d) where the min difficult is the max elements of the corresponding job starting index in max_arr.
    + also, note that we start from d-1 to n, as the earliest job starting index is d-1
  + we then traverse the DP array, column by column from column d-1 to column 1 (remember day 0 will not be used)
  + in addition to the previous prinicple to traverse starting index from day-1, we also need to consider the end index to traverse because
    + these day variables are in the middle of day range, we have to leave sufficient jobs to the days afer the current day, which defines the end index to be n-(d-day)-1. For example, if d = 3, day = 2, and we have 5 jobs, n = 5, then for day 2, we can only traverse job indexes from 2 to 3, because job with index 4, which is the last job, must be kept for day 3
    + define max_ele as the max element so far that is initialized to jobDifficult(i)
    + set up internal loop for j to travese from i (starting index) to n-(d-day)-1,here j traverse on the dimension of job index for the current day.
  + apply tranistion equation of rs = min(rs, max_ele(j) + dp(j+1)(day+1)) for j in range \[i, n-(d-day)-1\]
    + max_ele(j) is the max element in jobDifficult list within the range \[i, j\] (from starting index of i to the current j job\[j\], inclusive, if we assign job j to the current day). This max_ele(j) is the cost/job difficulty of the current single day
    + the total difficulty of the current day needs to include another component, which is the total difficulty of the following days, which is dp(j+1)(day+1) 
    + this seems to be different than the commonly DP pattern that we either take or not take an element, but actually, they are the same. We will have to take at least on job for the current day, that is where we start the j loop from i as the starting index, we calculate the cost, we then test the cost of including the following day, and compare them to the current cost, which dose not include these days, and return the min cost for the current daty

In [2]:
# top down implementation
from typing import List

class Solution:
    def minDifficulty(self, jobDifficulty: List[int], d: int) -> int:
        if not jobDifficulty or len(jobDifficulty) < d:
            return -1
        
        n = len(jobDifficulty)
        if d == n:
            return sum(jobDifficulty)
        
        max_arr = [0] * n
        max_ele = jobDifficulty[-1]
        for i in range(n-1, -1, -1):
            max_ele = max(max_ele, jobDifficulty[i])
            max_arr[i] = max_ele            
        
        @lru_cache(None)
        def find_min(start:int, remain_day: int) -> int:
            if n-start < remain_day or remain_day == 0:
                return -1
            
            if n-start == remain_day:
                return sum(jobDifficulty[start:])
            
            if remain_day == 1:
                return max_arr[start]
            
            max_ele = jobDifficulty[start]
            rs = float("inf")
            for i in range(start, n-remain_day+1):
                max_ele = max(max_ele, jobDifficulty[i])
                rs = min(rs, max_ele + find_min(i+1, remain_day-1))
            return rs
        
        return find_min(0, d)                
        

In [3]:
# bottom up implementation

from typing import List
class Solution:
    def minDifficulty(self, jobDifficulty: List[int], d: int) -> int:
        if not jobDifficulty or len(jobDifficulty) < d:
            return -1
        
        n = len(jobDifficulty)
        if d == n:
            return sum(jobDifficulty)
        
        dp = [[-1] * (d+1) for _ in range(n)]
        max_arr = [0] * n              
        max_ele = jobDifficulty[-1]
                   
        for i in range(n-1, -1, -1):
            max_ele = max(max_ele, jobDifficulty[i])
            max_arr[i] = max_ele            
        
        for i in range(d-1, n):
            dp[i][d] = max_arr[i]
            
        for day in range(d-1, 0, -1):
            for i in range(day-1, n):
                rs = float("inf")
                max_ele = jobDifficulty[i]
                for j in range(i, n-1):
                    max_ele = max(max_ele, jobDifficulty[j])
                    
                    rs = min(rs, max_ele + dp[j+1][day+1] )
                dp[i][day] = rs                
        
        return dp[0][1]         

### Leetcode 322 Coin Change
* overview
  + given an integer array coins representing coins of different denominations and an integer amount representing a total amount of money
  + return the fewest number of coins you need to make up that amount
* implementation
  + 1D DP problem. Establish an array with the number of elements equals to amount +1, with default element values of -1
  + the first element (index 0) has a value of 0, corresponding to the number of coins needed to get 0
  + traverse from 1 to amount, and each time, traverse all the coins to find all possible coin that can get one of the previous element with a value >-1
  + find the min of all these previous elements, and add one. This means that finding the fewest number of coins that can get one of the amount previous to the current amount by add on extra coin
  + return dp(amount)
* time complexity:
  + O(amount * len(coins))
* space complexity:
  + O(amount)

In [1]:
# bottom up
from typing import List
class Solution:
    def coinChange(self, coins: List[int], amount: int) -> int:
        if amount == 0:
            return 0
        
        dp = [-1] * (amount+1)
        dp[0] = 0
        
        for i in range(1, amount+1):
            pre = [dp[i-coin] for coin in coins if coin <= i and dp[i-coin] > -1]
            if pre:
                dp[i] = min(pre) + 1
                
        return dp[amount]        

# top down
from functools import lru_cache


class Solution:
    def coinChange(self, coins: List[int], amount: int) -> int:

        @lru_cache(None)
        def dfs(rem):
            if rem < 0:
                return -1
            if rem == 0:
                return 0
            min_cost = float('inf')
            for coin in coins:
                res = dfs(rem - coin)
                if res != -1:
                    min_cost = min(min_cost, res + 1)
            return min_cost if min_cost != float('inf') else -1

        return dfs(amount)

### Leetcode 139 Word break
* general overview
  + Given a string s and a dictionary of strings wordDict, return true if s can be segmented into a space-separated sequence of one or more dictionary words
  + Note that the same word in the dictionary may be reused multiple times in the segmentation
  + example: s = "leetcode", wordDict=\["leet", "code"\]
    + here s can be segmented into leet and code from wordDict
* algorithm
  + this is a DP problem, since
    + the final answer of the entire s string can be decomposed into sub-problems of substrings
    + the choice of how to segment the s string in index n will impact the segment of substrings after index n
  + state variable
    + the index of s string with true or false indicating if it is possible to express the substring up to that index by wrods from the wordDict
    + traverse i from 0 and check if s\[:i+1\] can be decomposed from words in wordDict and assign the results to dp(i)
    + return dp(-1)
  + recurrent function
    ```python
        for word in wordDict:
            if i >= len(word) and (i == len(word)-1 or dp[i-len(word)]):
                if s[i-len(word)+1:i+1] == word:
                    dp[i] = True
                    break
    ```                
* time complexity
  + O(nkl) where n, k and l are the length of string s, the number of words in wordDict, and the average length of the words in wordDict, respectively
* space complexity
  + O(n)

In [2]:
from typing import List

# bottom up implementation
class Solution:
    def wordBreak(self, s: str, wordDict: List[str]) -> bool:
        if not wordDict and not s:
            return True
        if not wordDict or not s:
            return False
        
        n = len(s)
        dp = [False] * n
        
        for i in range(n):
            for word in wordDict:
                if i >= len(word)-1 and (i==len(word) -1 or dp[i-len(word)]):
                    if s[i-len(word)+1: i+1] == word:
                        dp[i] = True
                        break
        return dp[-1]                  

# top down
class Solution:
    def wordBreak(self, s: str, wordDict: List[str]) -> bool:
        n = len(s)
        
        @lru_cache
        def dp(i: int) -> bool:
            if i < 0:
                return True
            
            for word in wordDict:
                l = len(word)
                if i >= l-1 and dp(i-l):
                    if s[i-l+1:i+1] == word:
                        return True
            return False
        
        return dp(n-1)
        